# IMPORT LIBRARIES

In [ ]:
import sys
sys.path.append("../")

import config
import MODEL
import UTILS

import os
import math
import numpy as np
import pandas as pd
from collections import Counter
import random

import torch
from torch.utils.data.dataset import Dataset
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

torch.manual_seed(0)

# HYPERPARAMETERS

In [ ]:
# TIME SERIES INFO
window = config.window
stride = config.stride

# CHANNELS INFO
dynamic_channels = config.dynamic_channels
static_channels = config.static_channels
output_channels = config.output_channels

# LABELS INFO
unknown = config.unknown

# MODEL INFO
model_name = "vae_ealstm"
code_dim = config.code_dim
# device = torch.device("cpu")
device = torch.device(config.device)
contrastive_weight = config.contrastive_weight
forward_weight = config.forward_weight
sum_weight = contrastive_weight+forward_weight

# TRAIN INFO
train = config.train
batch_size = config.batch_size
epochs = config.epochs
learning_rate = config.learning_rate
train_percent = config.train_percent

print("Hyperparameters:{}".format(model_name))
print("window : {}".format(window))
print("dynamic_channels : {}".format(dynamic_channels))
print("static_channels : {}".format(static_channels))
print("output_channels : {}".format(output_channels))
print("unknown : {}".format(unknown))
print("model_name : {}".format(model_name))
print("code_dim : {}".format(code_dim))
print("device : {}".format(device))
print("contrastive_weight : {}".format(contrastive_weight))
print("forward_weight : {}".format(forward_weight))
print("train : {}".format(train))
print("batch_size : {}".format(batch_size))
print("epochs : {}".format(epochs))
print("learning_rate : {}".format(learning_rate))
print("train_percent : {}".format(train_percent))

# DEFINE DIRECTORIES

In [ ]:
PREPROCESSED_DIR = config.PREPROCESSED_DIR
RESULT_DIR = config.RESULT_DIR
MODEL_DIR = config.MODEL_DIR

# LOAD DATA

In [ ]:
def load_dataset(file):
	dataset = np.load(os.path.join(PREPROCESSED_DIR, "{}.npz".format(file)), allow_pickle=True)
	return dataset

def get_data(dataset, index, preprocessed=True):
	data = dataset["data"]
	if preprocessed:
		data = (data-dataset["train_data_means"])/dataset["train_data_stds"]
	data = np.nan_to_num(data, nan=unknown)
	data = data[dataset[index]]
	return data

# BUILD MODEL

In [ ]:
inverse_model = getattr(MODEL, "vae")(input_channels=len(dynamic_channels)+len(output_channels), code_dim=code_dim, output_channels=len(static_channels), device=device)
inverse_model = inverse_model.to(device)
pytorch_total_params = sum(p.numel() for p in inverse_model.parameters() if p.requires_grad)
print(inverse_model)
forward_model = getattr(MODEL, "ealstm")(input_dynamic_channels=len(dynamic_channels), input_static_channels=code_dim, code_dim=code_dim, output_channels=len(output_channels))
forward_model = forward_model.to(device)
pytorch_total_params += sum(p.numel() for p in forward_model.parameters() if p.requires_grad)
print(forward_model)
print("#Parameters:{}".format(pytorch_total_params))
mse_criterion = torch.nn.MSELoss(reduction="none")
kl_criterion = MODEL.KLLoss()
optimizer = torch.optim.Adam(list(inverse_model.parameters())+list(forward_model.parameters()), lr=learning_rate)

# TRAIN MODEL

In [ ]:
if train:

	train_loss = []
	valid_loss = []
	min_loss = 10000

	for epoch in range(1,epochs+1):

		# LOAD DATA
		file, index = "strided_train", "train_index"
		dataset = load_dataset(file)
		data = get_data(dataset, index)
		nodes, years, window, channels = data.shape
		# print(nodes, years, window, channels)

		# GET ANCHOR AND POSITIVE YEARS
		anchor_years = np.zeros((nodes, years))
		for node in range(nodes):
			anchor_years[node] = random.sample(range(years), years)
		anchor_years = anchor_years.astype(np.int64)
		positive_years = np.zeros((nodes, years))
		for node in range(nodes):
			positive_years[node] = random.sample(range(years), years)
		positive_years = positive_years.astype(np.int64)
		# print(anchor_years.shape, positive_years.shape)

		# GET TRAIN AND VALIDATION SET
		anchor_years_train = anchor_years[:,:int(train_percent*years)]
		anchor_years_valid = anchor_years[:,int(train_percent*years):]
		positive_years_train = positive_years[:,:int(train_percent*years)]
		positive_years_valid = positive_years[:,int(train_percent*years):]
		# print(anchor_years_train.shape, anchor_years_valid.shape, positive_years_train.shape, positive_years_valid.shape)

		# LOSS ON TRAIN SET
		inverse_model.train()
		forward_model.train()
		epoch_loss = 0
		epoch_kl_loss = 0
		epoch_forward_loss = 0
		for year in range(anchor_years_train.shape[1]):

			#Get (anchor,positive) Instances for each node
			anchor_data = data[np.arange(nodes), anchor_years_train[:, year]]
			positive_data = data[np.arange(nodes), positive_years_train[:, year]]
			# print(anchor_data.shape, positive_data.shape)

			# Remove pairs where (anchor,positive) years are same
			keep_idx = anchor_years_train[:, year] != positive_years_train[:, year]
			anchor_data = anchor_data[keep_idx]
			positive_data = positive_data[keep_idx]
			# print(anchor_data.shape, positive_data.shape)

			# Remove pairs where (anchor,positive) basins have unknown in streamflow
			keep_idx = np.zeros((anchor_data.shape[0], 2)).astype(bool)
			keep_idx[:,0] = (anchor_data[:,:,-1]!=unknown).all(axis=1)
			keep_idx[:,1] = (positive_data[:,:,-1]!=unknown).all(axis=1)
			keep_idx = keep_idx.all(axis=1)
			anchor_data = anchor_data[keep_idx]
			positive_data = positive_data[keep_idx]
			# print(anchor_data.shape, positive_data.shape)

			random_batches = random.sample(range(anchor_data.shape[0]),anchor_data.shape[0])
			for batch in range(math.ceil(anchor_data.shape[0]/batch_size)):

				optimizer.zero_grad()

				# GET BATCH DATA FOR INVERSE MODEL
				random_batch = random_batches[batch*batch_size:(batch+1)*batch_size]
				batch_anchor_data = torch.from_numpy(anchor_data[random_batch]).to(device)
				batch_input = batch_anchor_data[:,:,dynamic_channels+output_channels].to(device)
				batch_static = batch_anchor_data[:,0,static_channels].to(device)
				# print(batch_input.shape, batch_static.shape)

				# GET INVERSE OUTPUT
				batch_code_vec, batch_mu, batch_std, batch_static_pred, batch_input_pred = inverse_model(x=batch_input)
				# print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

				# GET BATCH DATA FOR FORWARD MODEL
				batch_positive_data = torch.from_numpy(positive_data[random_batch]).to(device)
				batch_dynamic_input = batch_positive_data[:, :, dynamic_channels].to(device)
				batch_static_input = batch_code_vec
				batch_label = batch_positive_data[:, :, output_channels].to(device)
				# print(batch_dynamic_input.shape, batch_static_input.shape, batch_label.shape)

				# GET FORWARD OUTPUT
				batch_label_pred = forward_model(x_dynamic=batch_dynamic_input.to(device), x_static=batch_static_input.to(device))
				# print(batch_label_pred.shape)

				# CALCULATE LOSS
				batch_kl_loss = kl_criterion(batch_code_vec, batch_mu, batch_std)										# KL LOSS
				batch_forward_loss = mse_criterion(batch_label, batch_label_pred)										# FORWARD LOSS (PER CHANNEL LOSS)
				mask = (batch_label!=unknown).float()																	# FORWARD LOSS (CREATE MASK)
				batch_forward_loss = batch_forward_loss * mask															# FORWARD LOSS (MULTIPLY MASK)
				batch_forward_loss, mask = torch.sum(batch_forward_loss, dim=2), (torch.sum(mask, dim=2)>0).float()		# FORWARD LOSS (PER INSTANCE LOSS)
				batch_forward_loss = torch.sum(batch_forward_loss)/torch.sum(mask)										# FORWARD LOSS (MEAN SEQUENCE LOSS)
				batch_loss = (contrastive_weight*batch_kl_loss + forward_weight*batch_forward_loss)/sum_weight
				# print(batch_loss.shape, batch_loss)

				# LOSS BACKPROPOGATE
				batch_loss.backward()
				optimizer.step()

				# AGGREGATE LOSS
				epoch_loss += batch_loss.item()
				epoch_kl_loss += batch_kl_loss.item()
				epoch_forward_loss += batch_forward_loss.item()

		epoch_loss /= ((batch+1)*(year+1))
		epoch_kl_loss /= ((batch+1)*(year+1))
		epoch_forward_loss /= ((batch+1)*(year+1))
		print('Epoch:{}\tTrain Loss:{:.4f}\tKL Loss:{:.4f}\tForward Loss:{:.4f}'.format(epoch, epoch_loss, epoch_kl_loss, epoch_forward_loss), end="\t")
		train_loss.append(epoch_loss)

		# LOSS ON VALIDATION SET
		inverse_model.eval()
		forward_model.eval()
		epoch_loss = 0
		epoch_kl_loss = 0
		epoch_forward_loss = 0
		for year in range(anchor_years_valid.shape[1]):

			#Get (anchor,positive) Instances for each node
			anchor_data = data[np.arange(nodes), anchor_years_valid[:, year]]
			positive_data = data[np.arange(nodes), positive_years_valid[:, year]]
			# print(anchor_data.shape, positive_data.shape)

			# Remove pairs where (anchor,positive) years are same
			keep_idx = anchor_years_valid[:, year] != positive_years_valid[:, year]
			anchor_data = anchor_data[keep_idx]
			positive_data = positive_data[keep_idx]
			# print(anchor_data.shape, positive_data.shape)

			# Remove pairs where (anchor,positive) basins have unknown in streamflow
			keep_idx = np.zeros((anchor_data.shape[0], 2)).astype(bool)
			keep_idx[:,0] = (anchor_data[:,:,-1]!=unknown).all(axis=1)
			keep_idx[:,1] = (positive_data[:,:,-1]!=unknown).all(axis=1)
			keep_idx = keep_idx.all(axis=1)
			anchor_data = anchor_data[keep_idx]
			positive_data = positive_data[keep_idx]
			# print(anchor_data.shape, positive_data.shape)

			random_batches = random.sample(range(anchor_data.shape[0]),anchor_data.shape[0])
			for batch in range(math.ceil(anchor_data.shape[0]/batch_size)):

				# GET BATCH DATA FOR INVERSE MODEL
				random_batch = random_batches[batch*batch_size:(batch+1)*batch_size]
				batch_anchor_data = torch.from_numpy(anchor_data[random_batch]).to(device)
				batch_input = batch_anchor_data[:,:,dynamic_channels+output_channels].to(device)
				batch_static = batch_anchor_data[:,0,static_channels].to(device)
				# print(batch_input.shape, batch_static.shape)

				# GET INVERSE OUTPUT
				batch_code_vec, batch_mu, batch_std, batch_static_pred, batch_input_pred = inverse_model(x=batch_input)
				# print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

				# GET BATCH DATA FOR FORWARD MODEL
				batch_positive_data = torch.from_numpy(positive_data[random_batch]).to(device)
				batch_dynamic_input = batch_positive_data[:, :, dynamic_channels].to(device)
				batch_static_input = batch_code_vec
				batch_label = batch_positive_data[:, :, output_channels].to(device)
				# print(batch_dynamic_input.shape, batch_static_input.shape, batch_label.shape)

				# GET FORWARD OUTPUT
				batch_label_pred = forward_model(x_dynamic=batch_dynamic_input.to(device), x_static=batch_static_input.to(device))
				# print(batch_label_pred.shape)

				# CALCULATE LOSS
				batch_kl_loss = kl_criterion(batch_code_vec, batch_mu, batch_std)										# KL LOSS
				batch_forward_loss = mse_criterion(batch_label, batch_label_pred)										# FORWARD LOSS (PER CHANNEL LOSS)
				mask = (batch_label!=unknown).float()																	# FORWARD LOSS (CREATE MASK)
				batch_forward_loss = batch_forward_loss * mask															# FORWARD LOSS (MULTIPLY MASK)
				batch_forward_loss, mask = torch.sum(batch_forward_loss, dim=2), (torch.sum(mask, dim=2)>0).float()		# FORWARD LOSS (PER INSTANCE LOSS)
				batch_forward_loss = torch.sum(batch_forward_loss)/torch.sum(mask)										# FORWARD LOSS (MEAN SEQUENCE LOSS)
				batch_loss = (contrastive_weight*batch_kl_loss + forward_weight*batch_forward_loss)/sum_weight
				# print(batch_loss.shape, batch_loss)

				# AGGREGATE LOSS
				epoch_loss += batch_loss.item()
				epoch_kl_loss += batch_kl_loss.item()
				epoch_forward_loss += batch_forward_loss.item()

		epoch_loss /= ((batch+1)*(year+1))
		epoch_kl_loss /= ((batch+1)*(year+1))
		epoch_forward_loss /= ((batch+1)*(year+1))
		print('Valid Loss:{:.4f}\tKL Loss:{:.4f}\tForward Loss:{:.4f}\tMin Loss:{:.4f}'.format(epoch_loss, epoch_kl_loss, epoch_forward_loss, min_loss))
		valid_loss.append(epoch_loss)
		if min_loss>epoch_loss:
			min_loss = epoch_loss
			torch.save(inverse_model.state_dict(), os.path.join(MODEL_DIR, "{}_inverse".format(model_name)))
			torch.save(forward_model.state_dict(), os.path.join(MODEL_DIR, "{}_forward".format(model_name)))

	# PLOT LOSS
	fig = plt.figure(figsize=(10,10))
	ax1 = fig.add_subplot(111)
	ax1.set_xlabel("#Epoch", fontsize=50)

	# PLOT TRAIN LOSS
	lns1 = ax1.plot(train_loss, color='red', marker='o', linewidth=4, label="TRAIN LOSS")

	# PLOT VALIDATION SCORE
	ax2 = ax1.twinx()
	lns2 = ax2.plot(valid_loss, color='blue', marker='o', linewidth=4, label="VAL LOSS")

	# added these three lines
	lns = lns1+lns2
	labs = [l.get_label() for l in lns]
	ax1.legend(lns, labs, loc="upper right", fontsize=40, frameon=False)

	plt.tight_layout(pad=0.0,h_pad=0.0,w_pad=0.0)
	plt.savefig(os.path.join(RESULT_DIR, "{}_SCORE.pdf".format(model_name)), format = "pdf")
	plt.close()

# LOAD MODEL

In [ ]:
inverse_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, "{}_inverse".format(model_name))))
inverse_model.eval()
forward_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, "{}_forward".format(model_name))))
forward_model.eval()

# TEST MODEL (AVERAGED EMBEDDINGS)

## IN DISTRIBUTION

In [ ]:
file, index = "strided_train", "train_index"
dataset = load_dataset(file)
data = get_data(dataset, index)
nodes, years, window, channels = data.shape
# print(nodes, years, window, channels)

dataset_code = unknown*np.ones((nodes, years, code_dim), dtype=np.float32)
for year in range(years):

	#Get instances for each node
	node_data = data[np.arange(nodes), year]
	# print(node_data.shape)

	# Remove pairs where node have unknown in streamflow
	keep_idx = np.zeros((node_data.shape[0], 1)).astype(bool)
	keep_idx[:,0] = (node_data[:,:,-1]!=unknown).all(axis=1)
	keep_idx = keep_idx.all(axis=1)
	node_data = node_data[keep_idx]
	# print(node_data.shape)

	for batch in range(math.ceil(node_data.shape[0]/batch_size)):

		# GET BATCH DATA FOR INVERSE MODEL
		batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_input = batch_data[:,:,dynamic_channels+output_channels]
		batch_static = batch_data[:,0,static_channels]
		# print(batch_input.shape, batch_static.shape)

		# GET INVERSE OUTPUT
		batch_code_vec, batch_mu, batch_std, batch_static_pred, batch_input_pred = inverse_model(x=batch_input)
		# print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

		# STORE OUTPUT
		dataset_code[batch*batch_size:(batch+1)*batch_size, year] = batch_code_vec.detach().cpu().numpy()
dataset_code = np.mean(dataset_code, axis=1)

file, index = "strided_test", "train_index"
dataset = load_dataset(file)
data = get_data(dataset, index)
nodes, years, window, channels = data.shape
# print(nodes, years, window, channels)

dataset_true = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
dataset_pred = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
for year in range(years):

	#Get instance for each node
	node_data = data[np.arange(nodes), year]
	# print(node_data.shape)

	for batch in range(math.ceil(nodes/batch_size)):

		# GET BATCH DATA FOR FORWARD MODEL
		batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_dynamic_input = batch_data[:, :, dynamic_channels].to(device)
		batch_static_input = torch.from_numpy(dataset_code[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_label = batch_data[:, :, output_channels].to(device)
		# print(batch_dynamic_input.shape, batch_static_input.shape, batch_label.shape)

		# GET FORWARD OUTPUT
		batch_label_pred = forward_model(x_dynamic=batch_dynamic_input.to(device), x_static=batch_static_input.to(device))
		# print(batch_label_pred.shape)

		# STORE OUTPUT
		dataset_true[batch*batch_size:(batch+1)*batch_size, year] = batch_label.detach().cpu().numpy()
		dataset_pred[batch*batch_size:(batch+1)*batch_size, year] = batch_label_pred.detach().cpu().numpy()

dataset_true = (dataset_true*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
dataset_pred = (dataset_pred*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
dataset_true = UTILS.unstride_array(dataset_true)
dataset_pred = UTILS.unstride_array(dataset_pred)
dataset_true = dataset_true[:, stride:]
dataset_pred = dataset_pred[:, stride:]

per_sample_RMSE = UTILS.per_sample_RMSE(dataset_true, dataset_pred, unknown)
_, per_node_RMSE = UTILS.per_node_RMSE(dataset_true, dataset_pred, unknown)
per_sample_R2 = UTILS.per_sample_R2(dataset_true, dataset_pred, unknown)
_, per_node_R2 = UTILS.per_node_R2(dataset_true, dataset_pred, unknown)
print("Per Sample RMSE:{:.4f}\tPer Node RMSE:{:.4f}\tPer Sample R2:{:.4f}\tPer Node R2:{:.4f}".format(per_sample_RMSE, per_node_RMSE, per_sample_R2, per_node_R2))
np.save(os.path.join(RESULT_DIR, "{}_{}_{}".format(file, index, "true")), dataset_true)
np.save(os.path.join(RESULT_DIR, "{}_{}_{}_{}".format(file, index, model_name, "averaged")), dataset_pred)

## OUT DISTRIBUTION

In [ ]:
file, index = "strided_train", "test_index"
dataset = load_dataset(file)
data = get_data(dataset, index)
nodes, years, window, channels = data.shape
# print(nodes, years, window, channels)

dataset_code = unknown*np.ones((nodes, years, code_dim), dtype=np.float32)
for year in range(years):

	#Get instances for each node
	node_data = data[np.arange(nodes), year]
	# print(node_data.shape)

	# Remove pairs where node have unknown in streamflow
	keep_idx = np.zeros((node_data.shape[0], 1)).astype(bool)
	keep_idx[:,0] = (node_data[:,:,-1]!=unknown).all(axis=1)
	keep_idx = keep_idx.all(axis=1)
	node_data = node_data[keep_idx]
	# print(node_data.shape)

	for batch in range(math.ceil(node_data.shape[0]/batch_size)):

		# GET BATCH DATA FOR INVERSE MODEL
		batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_input = batch_data[:,:,dynamic_channels+output_channels]
		batch_static = batch_data[:,0,static_channels]
		# print(batch_input.shape, batch_static.shape)

		# GET INVERSE OUTPUT
		batch_code_vec, batch_mu, batch_std, batch_static_pred, batch_input_pred = inverse_model(x=batch_input)
		# print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

		# STORE OUTPUT
		dataset_code[batch*batch_size:(batch+1)*batch_size, year] = batch_code_vec.detach().cpu().numpy()
dataset_code = np.mean(dataset_code, axis=1)

file, index = "strided_test", "test_index"
dataset = load_dataset(file)
data = get_data(dataset, index)
nodes, years, window, channels = data.shape
# print(nodes, years, window, channels)

dataset_true = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
dataset_pred = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
for year in range(years):

	#Get instance for each node
	node_data = data[np.arange(nodes), year]
	# print(node_data.shape)

	for batch in range(math.ceil(nodes/batch_size)):

		# GET BATCH DATA FOR FORWARD MODEL
		batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_dynamic_input = batch_data[:, :, dynamic_channels].to(device)
		batch_static_input = torch.from_numpy(dataset_code[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_label = batch_data[:, :, output_channels].to(device)
		# print(batch_dynamic_input.shape, batch_static_input.shape, batch_label.shape)

		# GET FORWARD OUTPUT
		batch_label_pred = forward_model(x_dynamic=batch_dynamic_input.to(device), x_static=batch_static_input.to(device))
		# print(batch_label_pred.shape)

		# STORE OUTPUT
		dataset_true[batch*batch_size:(batch+1)*batch_size, year] = batch_label.detach().cpu().numpy()
		dataset_pred[batch*batch_size:(batch+1)*batch_size, year] = batch_label_pred.detach().cpu().numpy()

dataset_true = (dataset_true*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
dataset_pred = (dataset_pred*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
dataset_true = UTILS.unstride_array(dataset_true)
dataset_pred = UTILS.unstride_array(dataset_pred)
dataset_true = dataset_true[:, stride:]
dataset_pred = dataset_pred[:, stride:]

per_sample_RMSE = UTILS.per_sample_RMSE(dataset_true, dataset_pred, unknown)
_, per_node_RMSE = UTILS.per_node_RMSE(dataset_true, dataset_pred, unknown)
per_sample_R2 = UTILS.per_sample_R2(dataset_true, dataset_pred, unknown)
_, per_node_R2 = UTILS.per_node_R2(dataset_true, dataset_pred, unknown)
print("Per Sample RMSE:{:.4f}\tPer Node RMSE:{:.4f}\tPer Sample R2:{:.4f}\tPer Node R2:{:.4f}".format(per_sample_RMSE, per_node_RMSE, per_sample_R2, per_node_R2))
np.save(os.path.join(RESULT_DIR, "{}_{}_{}".format(file, index, "true")), dataset_true)
np.save(os.path.join(RESULT_DIR, "{}_{}_{}_{}".format(file, index, model_name, "averaged")), dataset_pred)

# TEST MODEL (YEARLY AVERAGED EMBEDDINGS I)

## IN DISTRIBUTION

In [ ]:
file, index = "strided_train", "train_index"
dataset = load_dataset(file)
data = get_data(dataset, index)
nodes, years, window, channels = data.shape
# print(nodes, years, window, channels)

dataset_code = np.nan*np.ones((nodes, years, code_dim), dtype=np.float32)
for year in range(years):

	#Get instances for each node
	node_data = data[np.arange(nodes), year]
	# print(node_data.shape)

	# Remove pairs where node have unknown in streamflow
	keep_idx = np.zeros((node_data.shape[0], 1)).astype(bool)
	keep_idx[:,0] = (node_data[:,:,-1]!=unknown).all(axis=1)
	keep_idx = keep_idx.all(axis=1)
	node_data = node_data[keep_idx]
	# print(node_data.shape)

	for batch in range(math.ceil(node_data.shape[0]/batch_size)):

		# GET BATCH DATA FOR INVERSE MODEL
		batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_input = batch_data[:,:,dynamic_channels+output_channels]
		batch_static = batch_data[:,0,static_channels]
		# print(batch_input.shape, batch_static.shape)

		# GET INVERSE OUTPUT
		batch_code_vec, batch_mu, batch_std, batch_static_pred, batch_input_pred = inverse_model(x=batch_input)
		# print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

		# STORE OUTPUT
		dataset_code[batch*batch_size:(batch+1)*batch_size, year] = batch_code_vec.detach().cpu().numpy()

file, index = "strided_test", "train_index"
dataset = load_dataset(file)
data = get_data(dataset, index)
nodes, years, window, channels = data.shape
# print(nodes, years, window, channels)

dataset_true = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
dataset_pred = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
for year in range(years):

	# COLLECT EMBEDDINGS
	year_code = np.nan*np.ones((nodes, code_dim), dtype=np.float32)

	#Get instance for each node
	node_data = data[np.arange(nodes), year]
	# print(node_data.shape)

	for batch in range(math.ceil(nodes/batch_size)):

		# GET BATCH DATA FOR FORWARD MODEL
		batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_dynamic_input = batch_data[:, :, dynamic_channels].to(device)
		batch_static_input = torch.from_numpy(np.nanmean(dataset_code, axis=1)[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_label = batch_data[:, :, output_channels].to(device)
		# print(batch_dynamic_input.shape, batch_static_input.shape, batch_label.shape)

		# GET FORWARD OUTPUT
		batch_label_pred = forward_model(x_dynamic=batch_dynamic_input.to(device), x_static=batch_static_input.to(device))
		# print(batch_label_pred.shape)

		# STORE FORWARD OUTPUT
		dataset_true[batch*batch_size:(batch+1)*batch_size, year] = batch_label.detach().cpu().numpy()
		dataset_pred[batch*batch_size:(batch+1)*batch_size, year] = batch_label_pred.detach().cpu().numpy()

		# GET BATCH DATA FOR INVERSE MODEL
		batch_input = batch_data[:,:,dynamic_channels+output_channels]
		batch_static = batch_data[:,0,static_channels]
		# print(batch_input.shape, batch_static.shape)

		# GET INVERSE OUTPUT
		batch_code_vec, batch_mu, batch_std, batch_static_pred, batch_input_pred = inverse_model(x=batch_input)
		# print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

		# STORE INVERSE OUTPUT
		year_code[batch*batch_size:(batch+1)*batch_size] = batch_code_vec.detach().cpu().numpy()

	dataset_code = np.concatenate((dataset_code, np.expand_dims(year_code, axis=1)), axis=1)

dataset_true = (dataset_true*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
dataset_pred = (dataset_pred*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
dataset_true = UTILS.unstride_array(dataset_true)
dataset_pred = UTILS.unstride_array(dataset_pred)
dataset_true = dataset_true[:, stride:]
dataset_pred = dataset_pred[:, stride:]

per_sample_RMSE = UTILS.per_sample_RMSE(dataset_true, dataset_pred, unknown)
_, per_node_RMSE = UTILS.per_node_RMSE(dataset_true, dataset_pred, unknown)
per_sample_R2 = UTILS.per_sample_R2(dataset_true, dataset_pred, unknown)
_, per_node_R2 = UTILS.per_node_R2(dataset_true, dataset_pred, unknown)
print("Per Sample RMSE:{:.4f}\tPer Node RMSE:{:.4f}\tPer Sample R2:{:.4f}\tPer Node R2:{:.4f}".format(per_sample_RMSE, per_node_RMSE, per_sample_R2, per_node_R2))
np.save(os.path.join(RESULT_DIR, "{}_{}_{}".format(file, index, "true")), dataset_true)
np.save(os.path.join(RESULT_DIR, "{}_{}_{}_{}".format(file, index, model_name, "yearly_averaged_I")), dataset_pred)

## OUT DISTRIBUTION

In [ ]:
file, index = "strided_train", "test_index"
dataset = load_dataset(file)
data = get_data(dataset, index)
nodes, years, window, channels = data.shape
# print(nodes, years, window, channels)

dataset_code = np.nan*np.ones((nodes, years, code_dim), dtype=np.float32)
for year in range(years):

	#Get instances for each node
	node_data = data[np.arange(nodes), year]
	# print(node_data.shape)

	# Remove pairs where node have unknown in streamflow
	keep_idx = np.zeros((node_data.shape[0], 1)).astype(bool)
	keep_idx[:,0] = (node_data[:,:,-1]!=unknown).all(axis=1)
	keep_idx = keep_idx.all(axis=1)
	node_data = node_data[keep_idx]
	# print(node_data.shape)

	for batch in range(math.ceil(node_data.shape[0]/batch_size)):

		# GET BATCH DATA FOR INVERSE MODEL
		batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_input = batch_data[:,:,dynamic_channels+output_channels]
		batch_static = batch_data[:,0,static_channels]
		# print(batch_input.shape, batch_static.shape)

		# GET INVERSE OUTPUT
		batch_code_vec, batch_mu, batch_std, batch_static_pred, batch_input_pred = inverse_model(x=batch_input)
		# print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

		# STORE OUTPUT
		dataset_code[batch*batch_size:(batch+1)*batch_size, year] = batch_code_vec.detach().cpu().numpy()

file, index = "strided_test", "test_index"
dataset = load_dataset(file)
data = get_data(dataset, index)
nodes, years, window, channels = data.shape
# print(nodes, years, window, channels)

dataset_true = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
dataset_pred = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
for year in range(years):

	# COLLECT EMBEDDINGS
	year_code = np.nan*np.ones((nodes, code_dim), dtype=np.float32)

	#Get instance for each node
	node_data = data[np.arange(nodes), year]
	# print(node_data.shape)

	for batch in range(math.ceil(nodes/batch_size)):

		# GET BATCH DATA FOR FORWARD MODEL
		batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_dynamic_input = batch_data[:, :, dynamic_channels].to(device)
		batch_static_input = torch.from_numpy(np.nanmean(dataset_code, axis=1)[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_label = batch_data[:, :, output_channels].to(device)
		# print(batch_dynamic_input.shape, batch_static_input.shape, batch_label.shape)

		# GET FORWARD OUTPUT
		batch_label_pred = forward_model(x_dynamic=batch_dynamic_input.to(device), x_static=batch_static_input.to(device))
		# print(batch_label_pred.shape)

		# STORE FORWARD OUTPUT
		dataset_true[batch*batch_size:(batch+1)*batch_size, year] = batch_label.detach().cpu().numpy()
		dataset_pred[batch*batch_size:(batch+1)*batch_size, year] = batch_label_pred.detach().cpu().numpy()

		# GET BATCH DATA FOR INVERSE MODEL
		batch_input = batch_data[:,:,dynamic_channels+output_channels]
		batch_static = batch_data[:,0,static_channels]
		# print(batch_input.shape, batch_static.shape)

		# GET INVERSE OUTPUT
		batch_code_vec, batch_mu, batch_std, batch_static_pred, batch_input_pred = inverse_model(x=batch_input)
		# print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

		# STORE INVERSE OUTPUT
		year_code[batch*batch_size:(batch+1)*batch_size] = batch_code_vec.detach().cpu().numpy()

	dataset_code = np.concatenate((dataset_code, np.expand_dims(year_code, axis=1)), axis=1)

dataset_true = (dataset_true*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
dataset_pred = (dataset_pred*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
dataset_true = UTILS.unstride_array(dataset_true)
dataset_pred = UTILS.unstride_array(dataset_pred)
dataset_true = dataset_true[:, stride:]
dataset_pred = dataset_pred[:, stride:]

per_sample_RMSE = UTILS.per_sample_RMSE(dataset_true, dataset_pred, unknown)
_, per_node_RMSE = UTILS.per_node_RMSE(dataset_true, dataset_pred, unknown)
per_sample_R2 = UTILS.per_sample_R2(dataset_true, dataset_pred, unknown)
_, per_node_R2 = UTILS.per_node_R2(dataset_true, dataset_pred, unknown)
print("Per Sample RMSE:{:.4f}\tPer Node RMSE:{:.4f}\tPer Sample R2:{:.4f}\tPer Node R2:{:.4f}".format(per_sample_RMSE, per_node_RMSE, per_sample_R2, per_node_R2))
np.save(os.path.join(RESULT_DIR, "{}_{}_{}".format(file, index, "true")), dataset_true)
np.save(os.path.join(RESULT_DIR, "{}_{}_{}_{}".format(file, index, model_name, "yearly_averaged_I")), dataset_pred)

# TEST MODEL (YEARLY AVERAGED EMBEDDINGS II)

## IN DISTRIBUTION

In [ ]:
file, index = "strided_train", "train_index"
dataset = load_dataset(file)
data = get_data(dataset, index)
nodes, years, window, channels = data.shape
# print(nodes, years, window, channels)

dataset_code = np.nan*np.ones((nodes, years, code_dim), dtype=np.float32)
for year in range(years):

	#Get instances for each node
	node_data = data[np.arange(nodes), year]
	# print(node_data.shape)

	# Remove pairs where node have unknown in streamflow
	keep_idx = np.zeros((node_data.shape[0], 1)).astype(bool)
	keep_idx[:,0] = (node_data[:,:,-1]!=unknown).all(axis=1)
	keep_idx = keep_idx.all(axis=1)
	node_data = node_data[keep_idx]
	# print(node_data.shape)

	for batch in range(math.ceil(node_data.shape[0]/batch_size)):

		# GET BATCH DATA FOR INVERSE MODEL
		batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_input = batch_data[:,:,dynamic_channels+output_channels]
		batch_static = batch_data[:,0,static_channels]
		# print(batch_input.shape, batch_static.shape)

		# GET INVERSE OUTPUT
		batch_code_vec, batch_mu, batch_std, batch_static_pred, batch_input_pred = inverse_model(x=batch_input)
		# print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

		# STORE OUTPUT
		dataset_code[batch*batch_size:(batch+1)*batch_size, year] = batch_code_vec.detach().cpu().numpy()
dataset_code = np.nanmean(dataset_code, axis=1)

file, index = "strided_test", "train_index"
dataset = load_dataset(file)
data = get_data(dataset, index)
nodes, years, window, channels = data.shape
# print(nodes, years, window, channels)

dataset_true = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
dataset_pred = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
for year in range(years):

	# COLLECT EMBEDDINGS
	year_code = np.nan*np.ones((nodes, code_dim), dtype=np.float32)

	#Get instance for each node
	node_data = data[np.arange(nodes), year]
	# print(node_data.shape)

	for batch in range(math.ceil(nodes/batch_size)):

		# GET BATCH DATA FOR FORWARD MODEL
		batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_dynamic_input = batch_data[:, :, dynamic_channels].to(device)
		batch_static_input = torch.from_numpy(dataset_code[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_label = batch_data[:, :, output_channels].to(device)
		# print(batch_dynamic_input.shape, batch_static_input.shape, batch_label.shape)

		# GET FORWARD OUTPUT
		batch_label_pred = forward_model(x_dynamic=batch_dynamic_input.to(device), x_static=batch_static_input.to(device))
		# print(batch_label_pred.shape)

		# STORE FORWARD OUTPUT
		dataset_true[batch*batch_size:(batch+1)*batch_size, year] = batch_label.detach().cpu().numpy()
		dataset_pred[batch*batch_size:(batch+1)*batch_size, year] = batch_label_pred.detach().cpu().numpy()

		# GET BATCH DATA FOR INVERSE MODEL
		batch_input = batch_data[:,:,dynamic_channels+output_channels]
		batch_static = batch_data[:,0,static_channels]
		# print(batch_input.shape, batch_static.shape)

		# GET INVERSE OUTPUT
		batch_code_vec, batch_mu, batch_std, batch_static_pred, batch_input_pred = inverse_model(x=batch_input)
		# print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

		# STORE INVERSE OUTPUT
		year_code[batch*batch_size:(batch+1)*batch_size] = batch_code_vec.detach().cpu().numpy()

	dataset_code = np.nanmean(np.concatenate((np.expand_dims(dataset_code, axis=1), np.expand_dims(year_code, axis=1)), axis=1), axis=1)

dataset_true = (dataset_true*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
dataset_pred = (dataset_pred*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
dataset_true = UTILS.unstride_array(dataset_true)
dataset_pred = UTILS.unstride_array(dataset_pred)
dataset_true = dataset_true[:, stride:]
dataset_pred = dataset_pred[:, stride:]

per_sample_RMSE = UTILS.per_sample_RMSE(dataset_true, dataset_pred, unknown)
_, per_node_RMSE = UTILS.per_node_RMSE(dataset_true, dataset_pred, unknown)
per_sample_R2 = UTILS.per_sample_R2(dataset_true, dataset_pred, unknown)
_, per_node_R2 = UTILS.per_node_R2(dataset_true, dataset_pred, unknown)
print("Per Sample RMSE:{:.4f}\tPer Node RMSE:{:.4f}\tPer Sample R2:{:.4f}\tPer Node R2:{:.4f}".format(per_sample_RMSE, per_node_RMSE, per_sample_R2, per_node_R2))
np.save(os.path.join(RESULT_DIR, "{}_{}_{}".format(file, index, "true")), dataset_true)
np.save(os.path.join(RESULT_DIR, "{}_{}_{}_{}".format(file, index, model_name, "yearly_averaged_II")), dataset_pred)

## OUT DISTRIBUTION

In [ ]:
file, index = "strided_train", "test_index"
dataset = load_dataset(file)
data = get_data(dataset, index)
nodes, years, window, channels = data.shape
# print(nodes, years, window, channels)

dataset_code = np.nan*np.ones((nodes, years, code_dim), dtype=np.float32)
for year in range(years):

	#Get instances for each node
	node_data = data[np.arange(nodes), year]
	# print(node_data.shape)

	# Remove pairs where node have unknown in streamflow
	keep_idx = np.zeros((node_data.shape[0], 1)).astype(bool)
	keep_idx[:,0] = (node_data[:,:,-1]!=unknown).all(axis=1)
	keep_idx = keep_idx.all(axis=1)
	node_data = node_data[keep_idx]
	# print(node_data.shape)

	for batch in range(math.ceil(node_data.shape[0]/batch_size)):

		# GET BATCH DATA FOR INVERSE MODEL
		batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_input = batch_data[:,:,dynamic_channels+output_channels]
		batch_static = batch_data[:,0,static_channels]
		# print(batch_input.shape, batch_static.shape)

		# GET INVERSE OUTPUT
		batch_code_vec, batch_mu, batch_std, batch_static_pred, batch_input_pred = inverse_model(x=batch_input)
		# print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

		# STORE OUTPUT
		dataset_code[batch*batch_size:(batch+1)*batch_size, year] = batch_code_vec.detach().cpu().numpy()
dataset_code = np.nanmean(dataset_code, axis=1)

file, index = "strided_test", "test_index"
dataset = load_dataset(file)
data = get_data(dataset, index)
nodes, years, window, channels = data.shape
# print(nodes, years, window, channels)

dataset_true = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
dataset_pred = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
for year in range(years):

	# COLLECT EMBEDDINGS
	year_code = np.nan*np.ones((nodes, code_dim), dtype=np.float32)

	#Get instance for each node
	node_data = data[np.arange(nodes), year]
	# print(node_data.shape)

	for batch in range(math.ceil(nodes/batch_size)):

		# GET BATCH DATA FOR FORWARD MODEL
		batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_dynamic_input = batch_data[:, :, dynamic_channels].to(device)
		batch_static_input = torch.from_numpy(dataset_code[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_label = batch_data[:, :, output_channels].to(device)
		# print(batch_dynamic_input.shape, batch_static_input.shape, batch_label.shape)

		# GET FORWARD OUTPUT
		batch_label_pred = forward_model(x_dynamic=batch_dynamic_input.to(device), x_static=batch_static_input.to(device))
		# print(batch_label_pred.shape)

		# STORE FORWARD OUTPUT
		dataset_true[batch*batch_size:(batch+1)*batch_size, year] = batch_label.detach().cpu().numpy()
		dataset_pred[batch*batch_size:(batch+1)*batch_size, year] = batch_label_pred.detach().cpu().numpy()

		# GET BATCH DATA FOR INVERSE MODEL
		batch_input = batch_data[:,:,dynamic_channels+output_channels]
		batch_static = batch_data[:,0,static_channels]
		# print(batch_input.shape, batch_static.shape)

		# GET INVERSE OUTPUT
		batch_code_vec, batch_mu, batch_std, batch_static_pred, batch_input_pred = inverse_model(x=batch_input)
		# print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

		# STORE INVERSE OUTPUT
		year_code[batch*batch_size:(batch+1)*batch_size] = batch_code_vec.detach().cpu().numpy()

	dataset_code = np.nanmean(np.concatenate((np.expand_dims(dataset_code, axis=1), np.expand_dims(year_code, axis=1)), axis=1), axis=1)

dataset_true = (dataset_true*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
dataset_pred = (dataset_pred*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
dataset_true = UTILS.unstride_array(dataset_true)
dataset_pred = UTILS.unstride_array(dataset_pred)
dataset_true = dataset_true[:, stride:]
dataset_pred = dataset_pred[:, stride:]

per_sample_RMSE = UTILS.per_sample_RMSE(dataset_true, dataset_pred, unknown)
_, per_node_RMSE = UTILS.per_node_RMSE(dataset_true, dataset_pred, unknown)
per_sample_R2 = UTILS.per_sample_R2(dataset_true, dataset_pred, unknown)
_, per_node_R2 = UTILS.per_node_R2(dataset_true, dataset_pred, unknown)
print("Per Sample RMSE:{:.4f}\tPer Node RMSE:{:.4f}\tPer Sample R2:{:.4f}\tPer Node R2:{:.4f}".format(per_sample_RMSE, per_node_RMSE, per_sample_R2, per_node_R2))
np.save(os.path.join(RESULT_DIR, "{}_{}_{}".format(file, index, "true")), dataset_true)
np.save(os.path.join(RESULT_DIR, "{}_{}_{}_{}".format(file, index, model_name, "yearly_averaged_II")), dataset_pred)

# TEST MODEL (YEARLY EMBEDDINGS)

## IN DISTRIBUTION

In [ ]:
file, index = "strided_train", "train_index"
dataset = load_dataset(file)
data = get_data(dataset, index)
nodes, years, window, channels = data.shape
# print(nodes, years, window, channels)

dataset_code = np.nan*np.ones((nodes, years, code_dim), dtype=np.float32)
for year in range(years):

	#Get instances for each node
	node_data = data[np.arange(nodes), year]
	# print(node_data.shape)

	# Remove pairs where node have unknown in streamflow
	keep_idx = np.zeros((node_data.shape[0], 1)).astype(bool)
	keep_idx[:,0] = (node_data[:,:,-1]!=unknown).all(axis=1)
	keep_idx = keep_idx.all(axis=1)
	node_data = node_data[keep_idx]
	# print(node_data.shape)

	for batch in range(math.ceil(node_data.shape[0]/batch_size)):

		# GET BATCH DATA FOR INVERSE MODEL
		batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_input = batch_data[:,:,dynamic_channels+output_channels]
		batch_static = batch_data[:,0,static_channels]
		# print(batch_input.shape, batch_static.shape)

		# GET INVERSE OUTPUT
		batch_code_vec, batch_mu, batch_std, batch_static_pred, batch_input_pred = inverse_model(x=batch_input)
		# print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

		# STORE OUTPUT
		dataset_code[batch*batch_size:(batch+1)*batch_size, year] = batch_code_vec.detach().cpu().numpy()
dataset_code = np.nanmean(dataset_code, axis=1)

file, index = "strided_test", "train_index"
dataset = load_dataset(file)
data = get_data(dataset, index)
nodes, years, window, channels = data.shape
# print(nodes, years, window, channels)

dataset_true = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
dataset_pred = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
for year in range(years):

	# COLLECT EMBEDDINGS
	year_code = np.nan*np.ones((nodes, code_dim), dtype=np.float32)

	#Get instance for each node
	node_data = data[np.arange(nodes), year]
	# print(node_data.shape)

	for batch in range(math.ceil(nodes/batch_size)):

		# GET BATCH DATA FOR FORWARD MODEL
		batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_dynamic_input = batch_data[:, :, dynamic_channels].to(device)
		batch_static_input = torch.from_numpy(dataset_code[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_label = batch_data[:, :, output_channels].to(device)
		# print(batch_dynamic_input.shape, batch_static_input.shape, batch_label.shape)

		# GET FORWARD OUTPUT
		batch_label_pred = forward_model(x_dynamic=batch_dynamic_input.to(device), x_static=batch_static_input.to(device))
		# print(batch_label_pred.shape)

		# STORE FORWARD OUTPUT
		dataset_true[batch*batch_size:(batch+1)*batch_size, year] = batch_label.detach().cpu().numpy()
		dataset_pred[batch*batch_size:(batch+1)*batch_size, year] = batch_label_pred.detach().cpu().numpy()

		# GET BATCH DATA FOR INVERSE MODEL
		batch_input = batch_data[:,:,dynamic_channels+output_channels]
		batch_static = batch_data[:,0,static_channels]
		# print(batch_input.shape, batch_static.shape)

		# GET INVERSE OUTPUT
		batch_code_vec, batch_mu, batch_std, batch_static_pred, batch_input_pred = inverse_model(x=batch_input)
		# print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

		# STORE INVERSE OUTPUT
		year_code[batch*batch_size:(batch+1)*batch_size] = batch_code_vec.detach().cpu().numpy()

	dataset_code = year_code

dataset_true = (dataset_true*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
dataset_pred = (dataset_pred*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
dataset_true = UTILS.unstride_array(dataset_true)
dataset_pred = UTILS.unstride_array(dataset_pred)
dataset_true = dataset_true[:, stride:]
dataset_pred = dataset_pred[:, stride:]

per_sample_RMSE = UTILS.per_sample_RMSE(dataset_true, dataset_pred, unknown)
_, per_node_RMSE = UTILS.per_node_RMSE(dataset_true, dataset_pred, unknown)
per_sample_R2 = UTILS.per_sample_R2(dataset_true, dataset_pred, unknown)
_, per_node_R2 = UTILS.per_node_R2(dataset_true, dataset_pred, unknown)
print("Per Sample RMSE:{:.4f}\tPer Node RMSE:{:.4f}\tPer Sample R2:{:.4f}\tPer Node R2:{:.4f}".format(per_sample_RMSE, per_node_RMSE, per_sample_R2, per_node_R2))
np.save(os.path.join(RESULT_DIR, "{}_{}_{}".format(file, index, "true")), dataset_true)
np.save(os.path.join(RESULT_DIR, "{}_{}_{}_{}".format(file, index, model_name, "yearly")), dataset_pred)

## OUT DISTRIBUTION

In [ ]:
file, index = "strided_train", "test_index"
dataset = load_dataset(file)
data = get_data(dataset, index)
nodes, years, window, channels = data.shape
# print(nodes, years, window, channels)

dataset_code = np.nan*np.ones((nodes, years, code_dim), dtype=np.float32)
for year in range(years):

	#Get instances for each node
	node_data = data[np.arange(nodes), year]
	# print(node_data.shape)

	# Remove pairs where node have unknown in streamflow
	keep_idx = np.zeros((node_data.shape[0], 1)).astype(bool)
	keep_idx[:,0] = (node_data[:,:,-1]!=unknown).all(axis=1)
	keep_idx = keep_idx.all(axis=1)
	node_data = node_data[keep_idx]
	# print(node_data.shape)

	for batch in range(math.ceil(node_data.shape[0]/batch_size)):

		# GET BATCH DATA FOR INVERSE MODEL
		batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_input = batch_data[:,:,dynamic_channels+output_channels]
		batch_static = batch_data[:,0,static_channels]
		# print(batch_input.shape, batch_static.shape)

		# GET INVERSE OUTPUT
		batch_code_vec, batch_mu, batch_std, batch_static_pred, batch_input_pred = inverse_model(x=batch_input)
		# print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

		# STORE OUTPUT
		dataset_code[batch*batch_size:(batch+1)*batch_size, year] = batch_code_vec.detach().cpu().numpy()
dataset_code = np.nanmean(dataset_code, axis=1)

file, index = "strided_test", "test_index"
dataset = load_dataset(file)
data = get_data(dataset, index)
nodes, years, window, channels = data.shape
# print(nodes, years, window, channels)

dataset_true = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
dataset_pred = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
for year in range(years):

	# COLLECT EMBEDDINGS
	year_code = np.nan*np.ones((nodes, code_dim), dtype=np.float32)

	#Get instance for each node
	node_data = data[np.arange(nodes), year]
	# print(node_data.shape)

	for batch in range(math.ceil(nodes/batch_size)):

		# GET BATCH DATA FOR FORWARD MODEL
		batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_dynamic_input = batch_data[:, :, dynamic_channels].to(device)
		batch_static_input = torch.from_numpy(dataset_code[batch*batch_size:(batch+1)*batch_size]).to(device)
		batch_label = batch_data[:, :, output_channels].to(device)
		# print(batch_dynamic_input.shape, batch_static_input.shape, batch_label.shape)

		# GET FORWARD OUTPUT
		batch_label_pred = forward_model(x_dynamic=batch_dynamic_input.to(device), x_static=batch_static_input.to(device))
		# print(batch_label_pred.shape)

		# STORE FORWARD OUTPUT
		dataset_true[batch*batch_size:(batch+1)*batch_size, year] = batch_label.detach().cpu().numpy()
		dataset_pred[batch*batch_size:(batch+1)*batch_size, year] = batch_label_pred.detach().cpu().numpy()

		# GET BATCH DATA FOR INVERSE MODEL
		batch_input = batch_data[:,:,dynamic_channels+output_channels]
		batch_static = batch_data[:,0,static_channels]
		# print(batch_input.shape, batch_static.shape)

		# GET INVERSE OUTPUT
		batch_code_vec, batch_mu, batch_std, batch_static_pred, batch_input_pred = inverse_model(x=batch_input)
		# print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

		# STORE INVERSE OUTPUT
		year_code[batch*batch_size:(batch+1)*batch_size] = batch_code_vec.detach().cpu().numpy()

	dataset_code = year_code

dataset_true = (dataset_true*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
dataset_pred = (dataset_pred*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
dataset_true = UTILS.unstride_array(dataset_true)
dataset_pred = UTILS.unstride_array(dataset_pred)
dataset_true = dataset_true[:, stride:]
dataset_pred = dataset_pred[:, stride:]

per_sample_RMSE = UTILS.per_sample_RMSE(dataset_true, dataset_pred, unknown)
_, per_node_RMSE = UTILS.per_node_RMSE(dataset_true, dataset_pred, unknown)
per_sample_R2 = UTILS.per_sample_R2(dataset_true, dataset_pred, unknown)
_, per_node_R2 = UTILS.per_node_R2(dataset_true, dataset_pred, unknown)
print("Per Sample RMSE:{:.4f}\tPer Node RMSE:{:.4f}\tPer Sample R2:{:.4f}\tPer Node R2:{:.4f}".format(per_sample_RMSE, per_node_RMSE, per_sample_R2, per_node_R2))
np.save(os.path.join(RESULT_DIR, "{}_{}_{}".format(file, index, "true")), dataset_true)
np.save(os.path.join(RESULT_DIR, "{}_{}_{}_{}".format(file, index, model_name, "yearly")), dataset_pred)